In [3]:
import os
from transformers import MarianMTModel, MarianTokenizer, PegasusForConditionalGeneration, PegasusTokenizer

In [ ]:
# 加载中文到英文的MarianMT翻译模型
model_name_translation = "Helsinki-NLP/opus-mt-zh-en"
tokenizer_translation = MarianTokenizer.from_pretrained(model_name_translation)
model_translation = MarianMTModel.from_pretrained(model_name_translation)

MarianMTModel.from_pretrained(model_name_translation) is responsible for loading the model weights into memory. It will trigger a `pytorch_model.bin` file to be stored `locally` on your computer if the process completes successfully.

If you are downloading a pre-trained model from Hugging Face's transformers library (or similar), the file is cached locally in your home directory `~/.cache/huggingface/transformers/`. Within this directory, subdirectories are created for each model based on its name or repository.

If you specify a custom path when downloading or saving the model (e.g., model.save_pretrained('custom_path')), the file will be stored in that directory.

Once downloaded, the file remains on your disk and is reused whenever you load the model again, unless you manually delete it.

Managing Storage:

- Clear Cache: You can clear unused models from the cache if you need to free up space:
bash
`huggingface-cli cache delete`

- Move to External Drive: You can also move the cache directory to an external drive or another location by setting the TRANSFORMERS_CACHE environment variable:
bash
`export TRANSFORMERS_CACHE=/path/to/your/custom/location`


In [ ]:
# 加载英文总结的Pegasus模型
model_name_summary = "google/pegasus-xsum"
tokenizer_summary = PegasusTokenizer.from_pretrained(model_name_summary)
model_summary = PegasusForConditionalGeneration.from_pretrained(model_name_summary)

In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained("google/pegasus-xsum")
print(f"Model cached at: {model.config.architectures}")